In [ ]:
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas

In [18]:
import openmeteo_requests
import requests_cache
from retry_requests import retry
import subprocess
import sys
from siphon.simplewebservice.ndbc import NDBC
import time
import requests, pandas as pd
from datetime import datetime, timedelta, timezone, time
import os,json
from pathlib import Path
import meteomatics.api as api
import matplotlib.pyplot as plt
import pytz, folium, gspread
from datetime import datetime
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
from requests.exceptions import HTTPError  # Importer HTTPError
import sys
from urllib.error import HTTPError
from sklearn.preprocessing import StandardScaler


def install_and_import(*packages):
    for package in packages:
        try:
            # Tenter d'importer chaque bibliothèque
            globals()[package] = __import__(package)
            print(f"Le package '{package}' est déjà installé et importé.")
        except ImportError:
            # Si l'import échoue, installer la bibliothèque
            print(f"Le package '{package}' n'est pas installé. Installation en cours...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            # Ensuite, essayer à nouveau d'importer la bibliothèque
            globals()[package] = __import__(package)
            print(f"Le package '{package}' a été installé et importé.")
 
%pip install pyowm
# Utilisation de la fonction pour installer et importer plusieurs packages
install_and_import("dask", "pyowm", "ndbc_api", "plotly", "geopy")
import plotly.express as px
import re

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Le package 'dask' est déjà installé et importé.
Le package 'pyowm' est déjà installé et importé.
Le package 'ndbc_api' est déjà installé et importé.
Le package 'plotly' est déjà installé et importé.
Le package 'geopy' est déjà installé et importé.


In [5]:
# Définir les scopes requis pour accéder à Google Sheets et Google Drive
scopes = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/drive.file',
    'https://www.googleapis.com/auth/drive.appdata','https://www.googleapis.com/auth/drive.appfolder'
]

# Chemin vers le fichier de crédentials JSON
path_to_google_cred = "google_sheets_api.json"

# Créer les crédentials à partir du fichier de service account
creds = Credentials.from_service_account_file(path_to_google_cred, scopes=scopes)

# Autoriser l'accès en utilisant les crédentials
client = gspread.authorize(creds)

# ID de la feuille Google Sheets
sheet_id = "1rXyD4WW5DcrO2iCywfwLbqps3Z-RumxQH9hJofhVM24/"

# Ouvrir la feuille Google Sheets en utilisant son ID
spreadsheet = client.open_by_key(sheet_id)

worksheets = spreadsheet.worksheets()

drive_service = build('drive', 'v3', credentials=creds)

In [6]:
def drop_columns_if_exist(df, columns_to_drop):

    existing_columns = [col for col in columns_to_drop if col in df.columns]
    return df.drop(columns=existing_columns)

Get Data from APIs

Marine API

In [7]:
from ndbc_api import NdbcApi

# get all stations and some metadata as a Pandas DataFrame
stations_df = NdbcApi().stations()

stations_df.head()

,Station,Lat,Lon,Elevation,Name,Owner,Program,Type,Includes Meteorology,Includes Currents,Includes Water Quality,DART Program
0,0y2w3,44.794,-87.313,179.0,"Sturgeon Bay CG Station, WI",U.S.C.G. Marine Reporting Stations,IOOS Partners,fixed,False,False,False,False
1,13001,12.000,-23.000,0.0,NE Extension,Prediction and Research Moored Array in the At...,International Partners,buoy,True,False,False,False
2,13002,21.000,-23.000,0.0,NE Extension,Prediction and Research Moored Array in the At...,International Partners,buoy,True,False,False,False
3,13008,15.000,-38.000,0.0,Reggae,Prediction and Research Moored Array in the At...,International Partners,buoy,True,False,False,False
4,13009,8.000,-38.000,0.0,Lambada,Prediction and Research Moored Array in the At...,International Partners,buoy,False,False,False,False


Get Nearest

In [16]:
latitude = 11.2404
longitude = -74.211

nearest = NdbcApi().nearest_station(lat=latitude, lon=longitude)
nearest

for index, row in stations_df.iterrows():
    if row['Station'] == '42058':
        lat_buoy_near_SM = round(row['Lat'],2)
        lon_buoy_near_SM = round(row['Lon'],2)
df_marine = NDBC.realtime_observations(nearest)

print(f"The nearest buoy from Santa Marta is the buoy '{nearest}', lat {lat_buoy_near_SM} and lon {lon_buoy_near_SM}. \n It has {df_marine.shape[0]} rows")
df_marine.head()

The nearest buoy from Santa Marta is the buoy '42058', lat 14.51 and lon -75.15. 
 It has 6534 rows


,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,visibility,3hr_pressure_tendency,water_level_above_mean,time
0,80.0,9.0,11.0,1.9,8.0,4.8,87.0,1011.8,27.0,26.9,22.0,NaN,NaN,NaN,2025-02-20 20:50:00+00:00
1,80.0,9.0,11.0,NaN,NaN,NaN,NaN,1011.8,27.1,26.9,22.2,NaN,NaN,NaN,2025-02-20 20:40:00+00:00
2,90.0,9.0,11.0,NaN,NaN,NaN,NaN,1011.8,27.1,26.9,22.2,NaN,NaN,NaN,2025-02-20 20:30:00+00:00
3,80.0,9.0,11.0,1.8,7.0,4.8,72.0,1011.8,27.0,26.9,22.0,NaN,NaN,NaN,2025-02-20 20:20:00+00:00
4,80.0,9.0,11.0,1.8,NaN,4.8,72.0,1011.8,27.0,26.9,22.0,NaN,NaN,NaN,2025-02-20 20:10:00+00:00


In [10]:
df_marine['time'] = pd.to_datetime(df_marine['time'])

min_date = min(df_marine['time'])
max_date = max(df_marine['time'])

print(f"{min_date}\n {max_date}")

2025-01-06 00:00:00+00:00
 2025-02-20 19:30:00+00:00


In [11]:
from geopy.distance import geodesic

# Calculer la distance entre les deux bouées
coord_buoy1 = (latitude, longitude)  # Coordonnées de Buoy_51101
coord_buoy2 = (lat_buoy_near_SM, lon_buoy_near_SM)  # Coordonnées de la bouée la plus proche

# Calculer la distance en km
distance_km = geodesic(coord_buoy1, coord_buoy2).kilometers

# Créer la carte centrée sur la première bouée
m = folium.Map(location=[latitude, longitude], zoom_start=7)

# Ajouter les marqueurs
folium.Marker([latitude, longitude]).add_to(m)
folium.Marker([lat_buoy_near_SM, lon_buoy_near_SM], popup="Buoy closest to SM").add_to(m)

# Afficher la distance sur la carte
distance_text = f"Distance from Santa Marta: {distance_km:.2f} km"
folium.Marker([lat_buoy_near_SM, lon_buoy_near_SM], tooltip=distance_text).add_to(m)

# Afficher la carte
m

In [12]:

"""

#### Prise en charge de plusieurs buoys à la fois

caribbean_df = stations_df[(stations_df['Lat'] >= 9) & 
                           (stations_df['Lat'] <= 25) & 
                           (stations_df['Lon'] >= -85) & 
                           (stations_df['Lon'] <= -60)]
caribbean_df.shape[0]

buoys_ids = []

for index, row in caribbean_df.iterrows():
    buoys_ids.append(row['Station'])  
print(len(buoys_ids))    

# Liste pour stocker les dataframes
all_dataframes = []

# Compteurs pour les bouées réussies et échouées
successful_buoy = 0
failed_buoy = 0

# Parcours de chaque bouée
for buoy_id in buoys_ids:
    try:
        # Récupère les observations en temps réel pour chaque bouée
        df_buoy = NDBC.realtime_observations(buoy_id)

        # Ajoute le dataframe à la liste
        all_dataframes.append(df_buoy)
        successful_buoy += 1

        # Efface et met à jour la ligne des bouées réussies
        print(f"\r✅ Bouées réussies : {successful_buoy}", end='', flush=True)

    except Exception:
        # Incrémente le compteur des échouées pour toutes les erreurs
        failed_buoy += 1

        # Efface et met à jour la ligne des bouées échouées
        print(f"\r❌ Bouées échouées : {failed_buoy}", end='', flush=True)

# Affichage final propre avec un saut de ligne
print("\n")

# Concaténation des dataframes si disponibles
if all_dataframes:
    final_df = pd.concat(all_dataframes, ignore_index=True)
    print(f"✔️ Concaténation terminée. Nombre de lignes : {final_df.shape[0]} \n Nombre de colonnes : {final_df.shape[1]}")
    print(f"✅ Bouées réussies : {successful_buoy}", end='', flush=True)
    print(f"❌ Bouées échouées : {failed_buoy}", end='', flush=True)

else:
    print("⚠️ Aucune donnée récupérée.")
    
"""

'\n\n#### Prise en charge de plusieurs buoys à la fois\n\ncaribbean_df = stations_df[(stations_df[\'Lat\'] >= 9) & \n                           (stations_df[\'Lat\'] <= 25) & \n                           (stations_df[\'Lon\'] >= -85) & \n                           (stations_df[\'Lon\'] <= -60)]\ncaribbean_df.shape[0]\n\nbuoys_ids = []\n\nfor index, row in caribbean_df.iterrows():\n    buoys_ids.append(row[\'Station\'])  \nprint(len(buoys_ids))    \n\n# Liste pour stocker les dataframes\nall_dataframes = []\n\n# Compteurs pour les bouées réussies et échouées\nsuccessful_buoy = 0\nfailed_buoy = 0\n\n# Parcours de chaque bouée\nfor buoy_id in buoys_ids:\n    try:\n        # Récupère les observations en temps réel pour chaque bouée\n        df_buoy = NDBC.realtime_observations(buoy_id)\n\n        # Ajoute le dataframe à la liste\n        all_dataframes.append(df_buoy)\n        successful_buoy += 1\n\n        # Efface et met à jour la ligne des bouées réussies\n        print(f"\r✅ Bouées ré

In [13]:
def handle_null_values(df):
    # Calcul des pourcentages de valeurs manquantes
    missing_percent = round((df.isnull().sum() / len(df)) * 100, 2)
    missing_percent_str = missing_percent.astype(str) + '%'
    

    # Gestion des valeurs manquantes
    for column in df.columns:
        null_percentage = missing_percent[column]
        
        if null_percentage == 100:
            # Si 100% des valeurs sont manquantes, on supprime la colonne
            print(f"Supprime la colonne : {column} (100% de valeurs manquantes)")
            df = df.drop(column, axis=1)
        elif null_percentage > 50:
            # Si plus de 50% des valeurs sont manquantes, on supprime la colonne
            print(f"Supprime la colonne : {column} ({null_percentage}% de valeurs manquantes)")
            df = df.drop(column, axis=1)
        elif null_percentage > 0:
            # Si moins de 50% des valeurs sont manquantes, on impute avec la médiane
            print(f"Impute la colonne : {column} avec la médiane ({null_percentage}% de valeurs manquantes)")
            median_value = df[column].median()
            df[column] = df[column].fillna(median_value)
       
    # Retourner le DataFrame modifié
    return df

# Exemple d'appel de la fonction
df_marine = handle_null_values(df_marine)

print(df_marine.shape)
df_marine.head()

Impute la colonne : wind_direction avec la médiane (0.09% de valeurs manquantes)
Impute la colonne : wind_speed avec la médiane (0.09% de valeurs manquantes)
Impute la colonne : wind_gust avec la médiane (0.09% de valeurs manquantes)
Impute la colonne : wave_height avec la médiane (43.09% de valeurs manquantes)
Supprime la colonne : dominant_wave_period (66.99% de valeurs manquantes)
Impute la colonne : average_wave_period avec la médiane (43.1% de valeurs manquantes)
Impute la colonne : dominant_wave_direction avec la médiane (43.1% de valeurs manquantes)
Impute la colonne : pressure avec la médiane (0.05% de valeurs manquantes)
Impute la colonne : air_temperature avec la médiane (0.89% de valeurs manquantes)
Impute la colonne : water_temperature avec la médiane (1.16% de valeurs manquantes)
Impute la colonne : dewpoint avec la médiane (0.89% de valeurs manquantes)
Supprime la colonne : visibility (100% de valeurs manquantes)
Supprime la colonne : 3hr_pressure_tendency (83.27% de vale

,wind_direction,wind_speed,wind_gust,wave_height,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,time
0,90.0,10.0,12.0,2.4,5.8,80.0,1012.0,27.1,27.2,22.3,2025-02-20 19:30:00+00:00
1,80.0,10.0,11.0,1.7,4.7,73.0,1012.2,27.1,27.3,22.4,2025-02-20 19:20:00+00:00
2,80.0,10.0,12.0,1.7,4.7,73.0,1012.3,27.1,27.3,22.5,2025-02-20 19:10:00+00:00
3,80.0,10.0,12.0,2.4,5.8,80.0,1012.5,27.1,27.3,22.4,2025-02-20 19:00:00+00:00
4,80.0,10.0,12.0,1.8,4.9,70.0,1012.7,27.2,27.3,22.4,2025-02-20 18:50:00+00:00


In [14]:
# Convertir la colonne 'time' en datetime et la garder comme une colonne
df_marine['time'] = pd.to_datetime(df_marine['time'])

# Configurer 'time' comme index, puis resampler
df_marine.set_index('time', inplace=True)

# Resampler à l'heure et prendre la moyenne
df_marine = df_marine.resample('h').mean().round(2)

# Optionnel : Si tu veux garder 'time' comme une colonne, on peut le réinitialiser
df_marine.reset_index(inplace=True)

# Afficher le résultat
df_marine.head()

,time,wind_direction,wind_speed,wind_gust,wave_height,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint
0,2025-01-06 00:00:00+00:00,68.33,10.17,12.17,2.73,6.10,69.33,1013.38,27.43,27.30,23.60
1,2025-01-06 01:00:00+00:00,68.33,10.00,12.67,2.68,6.10,71.67,1014.00,27.37,27.20,23.72
2,2025-01-06 02:00:00+00:00,66.67,10.67,12.83,2.65,6.03,64.00,1014.08,27.52,27.20,23.65
3,2025-01-06 03:00:00+00:00,66.67,11.00,13.67,2.65,5.97,68.00,1013.52,27.40,27.12,23.80
4,2025-01-06 04:00:00+00:00,70.00,10.83,13.33,2.87,6.10,60.33,1013.00,27.38,27.22,23.72


open-meteo API

In [23]:
def meteo_api_request(coordinates, mode='historical', days=92, interval='hourly'):
    """
    Fonction pour récupérer les données météo depuis l'API Open-Meteo avec cache et réessayer en cas d'erreur.
    
    Paramètres:
        coordinates (list) : liste de coordonnées [latitude, longitude]
        mode (str) : intervalle de données ('forecast' ou 'historical', par défaut 'historical')
        days (int) : nombre de jours dans le passé ou dans le futur (par défaut : 92 pour historique)
        interval (str) : intervalle des données ('hourly' ou 'daily', par défaut 'hourly')

    Retourne :
        pd.DataFrame : un DataFrame avec les données météo
    """
    # Fonction utilitaire pour convertir les coordonnées avec ou sans suffixe (ex: '45.5W', '-45.5')
    def parse_coordinates(coord):
        # Vérifie si la coordonnée a un suffixe de direction (W, E, N, S)
        pattern = r"^([-+]?\d+(\.\d+)?)([NSEW]?)$"
        match = re.match(pattern, str(coord))
        if match:
            value = float(match.group(1))
            direction = match.group(3)
            
            if direction == 'W' or direction == 'S':
                value = -abs(value)  # Si direction est Ouest ou Sud, on inverse la valeur
            
            return value
        else:
            raise ValueError(f"Coordonnée invalide : {coord}")

    # Convertir les coordonnées
    latitude = parse_coordinates(coordinates[0])
    longitude = parse_coordinates(coordinates[1])

    # Setup de l'API client avec retry et cache
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    openmeteo = openmeteo_requests.Client(session=cache_session)

    url = "https://api.open-meteo.com/v1/forecast"
    
    # Paramètres de base
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "past_days": days if mode == 'historical' else None,  # Si historique, utiliser 'past_days'
        "forecast_days": days if mode == 'forecast' else None,  # Si forecast, utiliser 'forecast_days'
        "hourly" if interval.lower() == 'hourly' else "daily": [
            "temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "showers", 
            "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", 
            "cloud_cover_high", "visibility", "wind_speed_10m", "soil_temperature_0cm", "soil_moisture_0_to_1cm", 
            "is_day"
        ]
    }

    # Faire l'appel API
    responses = openmeteo.weather_api(url, params=params)

    # Traiter la réponse pour le premier emplacement
    response = responses[0]  # On prend la première réponse si plusieurs lieux sont fournis

    # Initialisation du dictionnaire pour les données à retourner
    data = {}

    # Processus des données en fonction du mode sélectionné
    if mode == 'historical':
        # Traitement pour données historiques
        if interval == 'hourly':
            hourly = response.Hourly()
            hourly_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=hourly.Interval()),
                    inclusive="left"
                )
            }
            hourly_variables = [
                "temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "showers", 
                "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", 
                "cloud_cover_high", "visibility", "wind_speed_10m", "soil_temperature_0cm", "soil_moisture_0_to_1cm", 
                "is_day"
            ]
            
            for i, var in enumerate(hourly_variables):
                hourly_data[var] = [round(value, 2) for value in hourly.Variables(i).ValuesAsNumpy()]

            return pd.DataFrame(hourly_data)

        elif interval == 'daily':
            daily = response.Daily()
            daily_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                    end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=daily.Interval()),
                    inclusive="left"
                )
            }
            daily_variables = [
                "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", 
                "sunrise", "sunset", "daylight_duration", "sunshine_duration", "uv_index_max", "uv_index_clear_sky_max", 
                "precipitation_sum", "rain_sum", "showers_sum", "precipitation_hours", "precipitation_probability_max", 
                "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum"
            ]
            
            for i, var in enumerate(daily_variables):
                daily_data[var] = [round(value, 2) for value in daily.Variables(i).ValuesAsNumpy()]

            return pd.DataFrame(daily_data)

    # If Forecast is chosen
    elif mode == 'forecast':
        # Traitement pour prévisions
        if interval == 'hourly':
            hourly = response.Hourly()
            hourly_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=hourly.Interval()),
                    inclusive="left"
                )
            }
            hourly_variables = [
                "temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "showers", 
                "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", 
                "cloud_cover_high", "visibility", "wind_speed_10m", "soil_temperature_0cm", "soil_moisture_0_to_1cm", 
                "is_day"
            ]
            
            for i, var in enumerate(hourly_variables):
                hourly_data[var] = [round(value, 2) for value in hourly.Variables(i).ValuesAsNumpy()]

            return pd.DataFrame(hourly_data)

        elif interval == 'daily':
            daily = response.Daily()
            daily_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                    end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=daily.Interval()),
                    inclusive="left"
                )
            }
            daily_variables = [
                "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", 
                "sunrise", "sunset", "daylight_duration", "sunshine_duration", "uv_index_max", "uv_index_clear_sky_max", 
                "precipitation_sum", "rain_sum", "showers_sum", "precipitation_hours", "precipitation_probability_max", 
                "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum"
            ]
            
            for i, var in enumerate(daily_variables):
                daily_data[var] = [round(value, 2) for value in daily.Variables(i).ValuesAsNumpy()]

            return pd.DataFrame(daily_data)

In [24]:
coordinates = [lat_buoy_near_SM, lon_buoy_near_SM]
df_meteo = meteo_api_request(coordinates=coordinates)
print(df_meteo.shape)
df_meteo.head()

(2376, 18)


,date,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,showers,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,2024-11-20 00:00:00+00:00,28.799999,74.0,23.700001,0.0,NaN,0.0,1010.000000,1010.000000,40.0,18.0,0.0,26.0,24140.0,28.459999,NaN,NaN,0.0
1,2024-11-20 01:00:00+00:00,28.700001,76.0,24.049999,0.1,NaN,0.1,1010.700012,1010.700012,17.0,10.0,0.0,5.0,22100.0,30.410000,NaN,NaN,0.0
2,2024-11-20 02:00:00+00:00,28.650000,77.0,24.219999,0.1,NaN,0.1,1011.099976,1011.099976,98.0,98.0,0.0,5.0,22040.0,31.309999,NaN,NaN,0.0
3,2024-11-20 03:00:00+00:00,28.350000,77.0,23.930000,1.1,NaN,1.1,1011.400024,1011.400024,100.0,100.0,0.0,5.0,23380.0,31.100000,NaN,NaN,0.0
4,2024-11-20 04:00:00+00:00,28.549999,75.0,23.680000,0.5,NaN,0.5,1011.500000,1011.500000,100.0,100.0,0.0,43.0,8680.0,30.889999,NaN,NaN,0.0


In [29]:
# Arrondir toutes les colonnes numériques à 2 décimales
df_meteo.loc[:, df_meteo.select_dtypes(include=['float32', 'float64']).columns] = df_meteo.select_dtypes(include=['float32', 'float64']).applymap(lambda x: round(x, 2))
df_meteo.head()

,date,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,showers,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,2024-11-20 00:00:00+00:00,28.80,74.0,23.70,0.0,NaN,0.0,1010.0,1010.0,40.0,18.0,0.0,26.0,24140.0,28.46,NaN,NaN,0.0
1,2024-11-20 01:00:00+00:00,28.70,76.0,24.05,0.1,NaN,0.1,1010.7,1010.7,17.0,10.0,0.0,5.0,22100.0,30.41,NaN,NaN,0.0
2,2024-11-20 02:00:00+00:00,28.65,77.0,24.22,0.1,NaN,0.1,1011.1,1011.1,98.0,98.0,0.0,5.0,22040.0,31.31,NaN,NaN,0.0
3,2024-11-20 03:00:00+00:00,28.35,77.0,23.93,1.1,NaN,1.1,1011.4,1011.4,100.0,100.0,0.0,5.0,23380.0,31.10,NaN,NaN,0.0
4,2024-11-20 04:00:00+00:00,28.55,75.0,23.68,0.5,NaN,0.5,1011.5,1011.5,100.0,100.0,0.0,43.0,8680.0,30.89,NaN,NaN,0.0


In [30]:
df_meteo = handle_null_values(df_meteo)
print(df_meteo.columns)
df_meteo.head()

Impute la colonne : rain avec la médiane (3.28% de valeurs manquantes)
Impute la colonne : soil_temperature_0cm avec la médiane (3.28% de valeurs manquantes)
Impute la colonne : soil_moisture_0_to_1cm avec la médiane (3.28% de valeurs manquantes)
Index(['date', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
       'precipitation', 'rain', 'showers', 'pressure_msl', 'surface_pressure',
       'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high',
       'visibility', 'wind_speed_10m', 'soil_temperature_0cm',
       'soil_moisture_0_to_1cm', 'is_day'],
      dtype='object')


,date,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,showers,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,2024-11-20 00:00:00+00:00,28.80,74.0,23.70,0.0,0.0,0.0,1010.0,1010.0,40.0,18.0,0.0,26.0,24140.0,28.46,28.2,0.0,0.0
1,2024-11-20 01:00:00+00:00,28.70,76.0,24.05,0.1,0.0,0.1,1010.7,1010.7,17.0,10.0,0.0,5.0,22100.0,30.41,28.2,0.0,0.0
2,2024-11-20 02:00:00+00:00,28.65,77.0,24.22,0.1,0.0,0.1,1011.1,1011.1,98.0,98.0,0.0,5.0,22040.0,31.31,28.2,0.0,0.0
3,2024-11-20 03:00:00+00:00,28.35,77.0,23.93,1.1,0.0,1.1,1011.4,1011.4,100.0,100.0,0.0,5.0,23380.0,31.10,28.2,0.0,0.0
4,2024-11-20 04:00:00+00:00,28.55,75.0,23.68,0.5,0.0,0.5,1011.5,1011.5,100.0,100.0,0.0,43.0,8680.0,30.89,28.2,0.0,0.0


In [ ]:
# Définir les paramètres
parameters = [
        'wind_speed_10m:ms',	
        'wind_dir_10m:d',
        'wind_gusts_10m_1h:ms',
        't_2m:C',
        't_max_2m_24h:C',
        't_min_2m_24h:C',
        'msl_pressure:hPa',
        'precip_1h:mm'
    ]

# Définir les coordonnées
coordinates = [(lat_buoy_near_SM, lon_buoy_near_SM)]

# Définir le modèle
model = 'mix'
# Début URL de l'API
DEFAULT_API_BASE_URL = "https://api.meteomatics.com"
enddate = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)
startdate = enddate - timedelta(days=1)
interval = timedelta(hours=1)


def meteo_api_request(parameters, coordinates, model, DEFAULT_API_BASE_URL, startdate, enddate, interval):
        try:
            # Appel de la fonction read_credentials pour obtenir les informations d'identification
            meteo_username, meteo_password = read_credentials()
            
            # Récupérer les données
            df_meteo = api.query_time_series(coordinates, startdate, enddate, interval, parameters, meteo_username, meteo_password, model=model)
            df_meteo.reset_index(inplace=True)
            return df_meteo

        except Exception as e:

            print(f"❌ Erreur lors de la récupération des données : {e}")
            return pd.DataFrame()

df_meteo=meteo_api_request(parameters, coordinates, model, DEFAULT_API_BASE_URL, startdate, enddate, interval)
df_meteo.columns

Index(['lat', 'lon', 'validdate', 'wind_speed_10m:ms', 'wind_dir_10m:d',
       'wind_gusts_10m_1h:ms', 't_2m:C', 't_max_2m_24h:C', 't_min_2m_24h:C',
       'msl_pressure:hPa', 'precip_1h:mm'],
      dtype='object')

In [ ]:
# Définir les paramètres
parameters = [
        'wind_speed_10m:ms',	
        'wind_dir_10m:d',
        'wind_gusts_10m_1h:ms',
        't_2m:C',
        't_max_2m_24h:C',
        't_min_2m_24h:C',
        'msl_pressure:hPa',
        'precip_1h:mm'
    ]

# Définir les coordonnées
coordinates = [(lat_buoy_near_SM, lon_buoy_near_SM)]

# Définir le modèle
model = 'mix'
# Début URL de l'API
DEFAULT_API_BASE_URL = "https://api.meteomatics.com"
enddate = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)
startdate = enddate - timedelta(days=1)
interval = timedelta(hours=1)


df_meteo=meteo_api_request(parameters, coordinates, model, DEFAULT_API_BASE_URL, startdate, enddate, interval)
df_meteo.columns

❌ Erreur lors de la récupération des données : Request with valid date 2025-02-18T19:00:00Z requires data access before 2025-02-19T00:00:00Z, which is not granted with this subscription type (e.g. trial). The valid time period for this account type starts at 2025-02-19T00:00:00Z and ends at 2025-03-02T00:00:00Z. Please contact sales@meteomatics.com and we are happy to provide an extended trial or an upgrade of your account.


RangeIndex(start=0, stop=0, step=1)

In [ ]:
df_meteo = drop_columns_if_exist(df_meteo, ['lat','lon'])
df_meteo.rename(columns={'validdate': 'time'}, inplace=True)
df_meteo['time'] = pd.to_datetime(df_meteo['time'])
df_marine['time'] = pd.to_datetime(df_marine['time'])

df_meteo.head()

,time,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_1h:ms,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,msl_pressure:hPa,precip_1h:mm
0,2025-02-19 19:00:00+00:00,8.4,98.6,11.6,27.0,27.7,26.5,1011.9,0.15
1,2025-02-19 20:00:00+00:00,7.1,101.2,11.3,26.9,27.7,26.5,1011.4,0.00
2,2025-02-19 21:00:00+00:00,6.9,98.6,10.9,26.8,27.7,26.5,1011.0,0.00
3,2025-02-19 22:00:00+00:00,6.9,92.8,11.0,26.9,27.7,26.5,1011.5,0.00
4,2025-02-19 23:00:00+00:00,7.8,90.4,11.0,26.8,27.7,26.5,1011.6,0.00


In [ ]:
df_marine.head()

,time,wind_direction,wind_speed,wind_gust,wave_height,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint
0,2025-01-06 00:00:00+00:00,68.33,10.17,12.17,2.73,6.10,69.33,1013.38,27.43,27.30,23.60
1,2025-01-06 01:00:00+00:00,68.33,10.00,12.67,2.68,6.10,71.67,1014.00,27.37,27.20,23.72
2,2025-01-06 02:00:00+00:00,66.67,10.67,12.83,2.65,6.03,64.00,1014.08,27.52,27.20,23.65
3,2025-01-06 03:00:00+00:00,66.67,11.00,13.67,2.65,5.97,68.00,1013.52,27.40,27.12,23.80
4,2025-01-06 04:00:00+00:00,70.00,10.83,13.33,2.87,6.10,60.33,1013.00,27.38,27.22,23.72


In [ ]:
df_meteo = handle_null_values(df_meteo)
print(df_meteo.shape)
df_meteo.head()

(25, 9)


,time,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_1h:ms,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,msl_pressure:hPa,precip_1h:mm
0,2025-02-19 19:00:00+00:00,8.4,98.6,11.6,27.0,27.7,26.5,1011.9,0.15
1,2025-02-19 20:00:00+00:00,7.1,101.2,11.3,26.9,27.7,26.5,1011.4,0.00
2,2025-02-19 21:00:00+00:00,6.9,98.6,10.9,26.8,27.7,26.5,1011.0,0.00
3,2025-02-19 22:00:00+00:00,6.9,92.8,11.0,26.9,27.7,26.5,1011.5,0.00
4,2025-02-19 23:00:00+00:00,7.8,90.4,11.0,26.8,27.7,26.5,1011.6,0.00


Rename Columns + Delete Useless Ones

In [ ]:
df_meteo.head()

,time,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_1h:ms,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,msl_pressure:hPa,precip_1h:mm
0,2025-02-19 19:00:00+00:00,8.4,98.6,11.6,27.0,27.7,26.5,1011.9,0.15
1,2025-02-19 20:00:00+00:00,7.1,101.2,11.3,26.9,27.7,26.5,1011.4,0.00
2,2025-02-19 21:00:00+00:00,6.9,98.6,10.9,26.8,27.7,26.5,1011.0,0.00
3,2025-02-19 22:00:00+00:00,6.9,92.8,11.0,26.9,27.7,26.5,1011.5,0.00
4,2025-02-19 23:00:00+00:00,7.8,90.4,11.0,26.8,27.7,26.5,1011.6,0.00


Merging Dataframes

In [ ]:
# Effectuer la jointure interne sur la colonne 'time'
df_merged = pd.merge(df_marine, df_meteo, on='time', how='inner')

# Afficher le résultat
df_merged.shape

(24, 19)

In [ ]:
df_merged.head()

,time,wind_direction,wind_speed,wind_gust,wave_height,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_1h:ms,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,msl_pressure:hPa,precip_1h:mm
0,2025-02-19 19:00:00+00:00,96.67,8.00,9.50,1.77,5.03,84.33,1010.95,26.93,27.63,23.33,8.4,98.6,11.6,27.0,27.7,26.5,1011.9,0.15
1,2025-02-19 20:00:00+00:00,95.00,7.33,9.00,1.73,5.03,80.67,1010.67,26.88,27.63,23.55,7.1,101.2,11.3,26.9,27.7,26.5,1011.4,0.00
2,2025-02-19 21:00:00+00:00,101.67,7.00,8.17,1.85,5.30,80.67,1010.58,26.78,27.60,23.53,6.9,98.6,10.9,26.8,27.7,26.5,1011.0,0.00
3,2025-02-19 22:00:00+00:00,95.00,7.17,8.33,1.83,5.23,78.33,1010.75,26.75,27.58,23.48,6.9,92.8,11.0,26.9,27.7,26.5,1011.5,0.00
4,2025-02-19 23:00:00+00:00,90.00,7.50,8.75,1.80,5.20,84.00,1011.12,26.78,27.50,23.50,7.8,90.4,11.0,26.8,27.7,26.5,1011.6,0.00


In [ ]:
null_count = df_merged.isna().sum()
null_count

time                       0
wind_direction             0
wind_speed                 0
wind_gust                  0
wave_height                0
average_wave_period        0
dominant_wave_direction    0
pressure                   0
air_temperature            0
water_temperature          0
dewpoint                   0
wind_speed_10m:ms          0
wind_dir_10m:d             0
wind_gusts_10m_1h:ms       0
t_2m:C                     0
t_max_2m_24h:C             0
t_min_2m_24h:C             0
msl_pressure:hPa           0
precip_1h:mm               0
dtype: int64

In [ ]:
# Convertir 'Datetime' en format datetime
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d/%m/%Y-%H:%M')

# Définir une fonction pour classifier les périodes de la journée
def get_day_period(hour):
    if 5 <= hour < 12:
        return 'Matin'
    elif 12 <= hour < 17:
        return 'Midi'
    elif 17 <= hour < 21:
        return 'Soir'
    else:
        return 'Nuit'

# Appliquer la fonction et créer la nouvelle colonne
df['Day Period'] = df['Datetime'].dt.hour.apply(get_day_period)

df = drop_columns(df, ['Datetime'])
# Voir le résultat
df.head()

,Wind Dir (°),Wind Speed (ms),Wind Gusts 1h (ms),Wave Height (m),Avg Wave Period (s),Dominant Wave Direction,Pressure,Air Temperature,Water Temperature,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_1h:ms,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,msl_pressure:hPa,precip_1h:mm,Day Period
0,96.67,8.00,9.50,1.77,5.03,84.33,1010.95,26.93,27.63,8.4,98.6,11.6,27.0,27.7,26.5,1011.9,0.15,Soir
1,95.00,7.33,9.00,1.73,5.03,80.67,1010.67,26.88,27.63,7.1,101.2,11.3,26.9,27.7,26.5,1011.4,0.00,Soir
2,101.67,7.00,8.17,1.85,5.30,80.67,1010.58,26.78,27.60,6.9,98.6,10.9,26.8,27.7,26.5,1011.0,0.00,Nuit
3,95.00,7.17,8.33,1.83,5.23,78.33,1010.75,26.75,27.58,6.9,92.8,11.0,26.9,27.7,26.5,1011.5,0.00,Nuit
4,90.00,7.50,8.75,1.80,5.20,84.00,1011.12,26.78,27.50,7.8,90.4,11.0,26.8,27.7,26.5,1011.6,0.00,Nuit


In [ ]:
marine_data_sheet = spreadsheet.get_worksheet(0)  # Accéder à la première feuille (gid=0)
marine_data = marine_data_sheet.get_all_records()  # Récupérer toutes les données de la feuille
print(marine_data)  # Afficher les données

APIError: APIError: [-1]: <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat 0% 0%/100% 100%;-m